In [60]:
import os
import json
import pickle
import numpy as np
import cv2
from keras.models import load_model
from tensorflow.keras.utils import img_to_array, load_img
from PIL import ImageFile


In [61]:
BASE_MODEL_PATH = "/.model"
TEST_DIR = "./test.csv"
PREDICT_DIR = "./pred_dir"
PICKLE_DIR = "./pickle_files"
JSON_DIR = "./json_files"

In [62]:
if not os.path.exists(JSON_DIR):
    os.makedirs(JSON_DIR)

In [63]:
BEST_MODEL = ".\model\cnn_non_batch_best_model.keras"
model = load_model(BEST_MODEL)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 128)    │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 512)    │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │    16,384,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,158,734 (130.31 MB)

 Trainable params: 17,079,366 (65.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,079,368 (65.15 MB)

In [64]:
import pandas as pd
data_test = pd.read_csv(os.path.join(TEST_DIR))
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   path        10000 non-null  object
 1   class_name  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [65]:
with open("D:\DistractedDriver\pickle_files\labels_list_cnn_non_batch.pkl","rb") as handle:
    labels_id = pickle.load(handle)
print(labels_id)

{'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


In [66]:
from tqdm import tqdm
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = load_img(img_path, target_size=(128,128))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True  
test_tensors = paths_to_tensor(data_test.iloc[:,0]).astype('float32')/255 - 0.5

100%|██████████| 10000/10000 [00:12<00:00, 817.94it/s]


In [67]:
#The 4 lines below are only for shallow (deep - 64, nonbatch-224)
# from tensorflow.keras.applications import VGG16  
# feature_extractor = VGG16(include_top=False, input_shape=(224, 224,3), weights='imagenet')
# test_features = feature_extractor.predict(test_tensors)
# ypred_test = model.predict(test_features)
ypred_test = model.predict(test_tensors,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels)

for i in range(data_test.shape[0]):
    data_test.iloc[i,1] = id_labels[ypred_class[i]]

313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step
{0: 'c0', 1: 'c1', 2: 'c2', 3: 'c3', 4: 'c4', 5: 'c5', 6: 'c6', 7: 'c7', 8: 'c8', 9: 'c9'}


In [68]:
#to create a human readable and understandable class_name 
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"


with open(os.path.join(JSON_DIR,'class_name_map.json'),'w') as secret_input:
    json.dump(class_name,secret_input,indent=4,sort_keys=True)

In [69]:
import shutil

# creating the prediction results for the image classification and shifting the predicted images to another folder
#with renamed filename having the class name predicted for that image using model
with open(os.path.join(JSON_DIR,'class_name_map.json')) as secret_input:
    info = json.load(secret_input)
PREDICT_DIR="./pred_dir/cnn_non_batch"
if not os.path.exists(PREDICT_DIR):
    os.makedirs(PREDICT_DIR)

for i in range(data_test.shape[0]):
    new_name = data_test.iloc[i,0].split("/")[-1].split(".")[0]+"_"+info[data_test.iloc[i,1]]+".jpg"
    shutil.copy(data_test.iloc[i,0],os.path.join(PREDICT_DIR,new_name))
    
#saving the model predicted results into a csv file
data_test.to_csv(os.path.join("./csv_files","cnn_non_batch.csv"),index=False)

In [70]:
import pandas as pd

path1 = "./csv_files/cnn_non_batch.csv"   # predicted
path2 = "./test_actual.csv"               # actual

pred = pd.read_csv(path1)
actual = pd.read_csv(path2)

# Merge on the "path" column
merged = pred.merge(actual, on="path", how="inner")

# Compare predicted vs actual
correct = (merged["class_name"] == merged["actual_name"]).sum()

print("Correct predictions:", correct)
print("Total images compared:", len(merged))
print("Accuracy:", correct / len(merged))


Correct predictions: 1000
Total images compared: 10000
Accuracy: 0.1
